In [96]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from aimanager.utils.utils import make_dir

import statsmodels.formula.api as smf
import scipy
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

file = '../../data/experiments/pilot_random1_player_round_slim.csv'
plot_folder = 'plots/distribution'

df = pd.read_csv(file)
df.head()

,session,global_group_id,episode,episode_id,experiment_name,round_number,participant_code,player_no_input,manager_no_input,player_id,contribution,punishment,payoff,common_good
0,mqqsqhav,mqqsqhav #3,1,112,trail_rounds_2,0,ac61awuk,0,1.0,0,15.0,0.0,22.0,69.0
1,mqqsqhav,mqqsqhav #9,1,124,trail_rounds_2,0,cqtrgylt,0,1.0,1,5.0,0.0,30.0,59.0
2,mqqsqhav,mqqsqhav #2,1,106,trail_rounds_2,0,dq7drghw,0,0.0,1,15.0,0.0,24.0,77.0
3,mqqsqhav,mqqsqhav #2,1,106,trail_rounds_2,0,u5q0svmp,0,0.0,0,9.0,0.0,30.0,77.0
4,mqqsqhav,mqqsqhav #3,1,112,trail_rounds_2,0,k5znx6rn,0,1.0,1,3.0,0.0,34.0,69.0


In [97]:
df['episode_id'].value_counts().value_counts()

32    82
64    49
16     2
28     1
24     1
Name: episode_id, dtype: int64

In [98]:
df = df.sort_values(['episode_id', 'player_id', 'round_number'])

df['values_per_episode'] = df.groupby('episode_id')['contribution'].transform('count')

df[['prev_punishment', 'prev_contribution']] = df.groupby(['episode_id', 'player_id'])[['punishment', 'contribution']].shift(1)
df[['prev2_punishment', 'prev2_contribution']] = df.groupby(['episode_id', 'player_id'])[['punishment', 'contribution']].shift(2)
df[['prev3_punishment', 'prev3_contribution']] = df.groupby(['episode_id', 'player_id'])[['punishment', 'contribution']].shift(3)


In [99]:
w = (
    df['values_per_episode'].isin([32, 64]) &
    (df['round_number'] > 1) &
    (df['prev_contribution'] != 20)
)


dfs = df[w].copy()

# m_str = [f'{a}*{b}' for a in label_str for b in label_str]


dfs['is_prev_punishment'] = dfs['prev_punishment'] > 0
dfs['prev_defect'] = (20 - dfs['prev_contribution']) 
dfs['prev2_defect'] = (20 - dfs['prev2_contribution']) 
dfs['ratio_prev_punishment'] = dfs['prev_punishment'] / dfs['prev_defect']
dfs['ratio_prev2_punishment'] = dfs['prev2_punishment'] / dfs['prev2_defect']
dfs['ratio_prev_punishment_bin'] = pd.cut(dfs['ratio_prev_punishment'], [-1, 0, 0.2, 0.4, 0.8, 1.2, 1000])
dfs['ratio_prev2_punishment_bin'] = pd.cut(dfs['ratio_prev2_punishment'], [-1, 0, 0.2, 0.4, 0.8, 1.2, 1000])

results = smf.glm(f"contribution ~ prev_contribution + C(ratio_prev_punishment_bin)", data=dfs).fit()
results_df = results.summary2().tables[1]

results_df

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,3.387981,0.211533,16.016313,9.830988e-58,2.973384,3.802579
"C(ratio_prev_punishment_bin)[T.Interval(0.0, 0.2, closed='right')]",-0.383831,0.295305,-1.299776,1.936779e-01,-0.962619,0.194957
"C(ratio_prev_punishment_bin)[T.Interval(0.2, 0.4, closed='right')]",0.112799,0.297149,0.379604,7.042392e-01,-0.469602,0.695200
"C(ratio_prev_punishment_bin)[T.Interval(0.4, 0.8, closed='right')]",1.060473,0.248322,4.270555,1.949872e-05,0.573771,1.547175
"C(ratio_prev_punishment_bin)[T.Interval(0.8, 1.2, closed='right')]",1.844814,0.272355,6.773571,1.256416e-11,1.311008,2.378619
"C(ratio_prev_punishment_bin)[T.Interval(1.2, 1000.0, closed='right')]",2.409930,0.324530,7.425904,1.120120e-13,1.773863,3.045998
prev_contribution,0.680551,0.014846,45.841925,0.000000e+00,0.651454,0.709647


In [100]:
results = smf.glm(f"contribution ~ prev_contribution + prev2_contribution + C(ratio_prev_punishment_bin) + C(ratio_prev2_punishment_bin)", data=dfs).fit()
results_df = results.summary2().tables[1]

results_df

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,1.422938,0.217515,6.541783,6.078967e-11,0.996616,1.849260
"C(ratio_prev_punishment_bin)[T.Interval(0.0, 0.2, closed='right')]",0.171241,0.288965,0.592601,5.534482e-01,-0.395120,0.737602
"C(ratio_prev_punishment_bin)[T.Interval(0.2, 0.4, closed='right')]",0.447114,0.294844,1.516442,1.294077e-01,-0.130770,1.024998
"C(ratio_prev_punishment_bin)[T.Interval(0.4, 0.8, closed='right')]",1.020545,0.282521,3.612274,3.035233e-04,0.466813,1.574277
"C(ratio_prev_punishment_bin)[T.Interval(0.8, 1.2, closed='right')]",1.856611,0.311912,5.952359,2.643052e-09,1.245275,2.467947
"C(ratio_prev_punishment_bin)[T.Interval(1.2, 1000.0, closed='right')]",2.325173,0.385808,6.026765,1.672744e-09,1.569003,3.081342
"C(ratio_prev2_punishment_bin)[T.Interval(0.0, 0.2, closed='right')]",0.076479,0.295642,0.258687,7.958766e-01,-0.502969,0.655927
"C(ratio_prev2_punishment_bin)[T.Interval(0.2, 0.4, closed='right')]",0.612110,0.294362,2.079447,3.757629e-02,0.035171,1.189048
"C(ratio_prev2_punishment_bin)[T.Interval(0.4, 0.8, closed='right')]",0.143230,0.278504,0.514282,6.070551e-01,-0.402628,0.689088
"C(ratio_prev2_punishment_bin)[T.Interval(0.8, 1.2, closed='right')]",0.754906,0.321122,2.350843,1.873093e-02,0.125520,1.384293
